This notebook is used to show the approach to the data deduplication using LSH (Locality Sensitive Hashing).

Installing dependencies

In [1]:
! pip install -r requirements.txt

In [1]:
from datasketch import MinHash, MinHashLSH
import os
import re
from tqdm import tqdm

Load a sample of the data

In [2]:
folder_path = './start_data'

files = os.listdir(folder_path)
print(f'Number of files: {len(files)}')
files[0]

Number of files: 48572


'116434150.txt'

In [24]:
def normalize_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def tokenize_text(text):
    return re.findall(r'\b\w+\b', text.lower())


def deduplicate_text_files_lsh(folder_path, threshold=0.5, num_perm=128, limit=None):
    files = os.listdir(folder_path)

    if limit:
        files = files[:limit]

    print(f"Processing {len(files)} files")

    duplicates = []
    unique_files = []

    original_to_duplicates = {}

    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    for file in tqdm(files):
        try:
            file_path = os.path.join(folder_path, file)
            file_content = open(file_path, 'r').read()
            minhash = MinHash(num_perm=num_perm)
            for word in tokenize_text(normalize_text(file_content)):
                minhash.update(word.encode('utf-8'))
        except Exception as e:
            print(f"Error processing file {file}: {e}")
            continue
        
        result = lsh.query(minhash)
        if result:
            canonical = result[0]
            original_to_duplicates.setdefault(canonical, []).append(file)
            duplicates.append(file)
        else:
            lsh.insert(file, minhash)
            unique_files.append(file)

    return duplicates, unique_files, original_to_duplicates

thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# for threshold in thresholds:
#     dupicates, unique_files, original_to_duplicates = deduplicate_text_files_lsh(folder_path, threshold=threshold)
#     print(f'Threshold: {threshold}')
# print(len(dupicates)/len(files) * 100)

In [25]:
dupicates, unique_files, original_to_duplicates = deduplicate_text_files_lsh(folder_path, threshold=0.5)

print(original_to_duplicates)

Processing 48572 files


100%|██████████| 48572/48572 [04:56<00:00, 163.55it/s]

{'116462316.txt': ['116432521.txt', '116464767.txt', '116504939.txt', '116432247.txt', '116464773.txt', '116431014.txt', '116454698.txt', '116468613.txt', '116467736.txt', '116433367.txt', '116494872.txt', '116436592.txt', '116496042.txt', '116452996.txt', '116466560.txt', '116462909.txt', '116430915.txt', '116442818.txt', '116469323.txt', '116463814.txt', '116449757.txt', '116431607.txt', '116469322.txt', '116441647.txt', '116464438.txt', '116437074.txt', '116427501.txt', '116430055.txt', '116467938.txt', '116453109.txt', '116427850.txt', '116431015.txt', '116462465.txt', '116464770.txt', '116469268.txt', '116435503.txt', '116464758.txt', '116427852.txt', '116440639.txt', '116429711.txt', '116429063.txt', '116426807.txt', '116430917.txt', '116426813.txt', '116437088.txt', '116431360.txt', '116446739.txt', '116456502.txt', '116436590.txt', '116449393.txt', '116427847.txt', '116446275.txt', '116462328.txt', '116469269.txt', '116431002.txt', '116464749.txt', '116437305.txt', '116470072.t

In [9]:
# The best setting looks likt 0.5 or 0.6. To be sure, we can run it again with 0.5 and then process unique files with 0.6
# And highlight the files that are duplicates to make sure they are duplicates

# Run it again with 0.5
dupicates, unique_files = deduplicate_text_files_lsh(folder_path, threshold=0.5)

# Process unique files with 0.6
# And highlight the files that are duplicates to make sure they are duplicates

# File to it's duplicates
duplicates = {}

lsh = MinHashLSH(threshold=0.6)
for file in tqdm(unique_files):
    file_path = os.path.join(folder_path, file)
    file_content = open(file_path, 'r').read()
    minhash = MinHash(num_perm=128)
    for word in tokenize_text(normalize_text(file_content)):
        minhash.update(word.encode('utf-8'))

    # Get duplicates
    result = lsh.query(minhash)
    if result:
        duplicates[file] = result
    
    lsh.insert(file, minhash)

# print(f'{file} has {len(result)} duplicates')

print(duplicates)



100%|██████████| 4967/4967 [00:35<00:00, 140.37it/s]

{'116450367.txt': ['116467252.txt'], '116436431.txt': ['116437101.txt'], '116458883.txt': ['116457024.txt'], '116468969.txt': ['116469121.txt'], '116426145.txt': ['116447545.txt'], '116428988.txt': ['116447961.txt'], '116438557.txt': ['116466158.txt'], '116466205.txt': ['116466206.txt'], '116486871.txt': ['116486906.txt'], '116429895.txt': ['116449811.txt'], '116436219.txt': ['116487615.txt'], '116467644.txt': ['116441268.txt'], '116505016.txt': ['116437102.txt'], '116468416.txt': ['116448514.txt'], '116471810.txt': ['116456311.txt'], '116449753.txt': ['116449756.txt'], '116429162.txt': ['116459181.txt'], '116436056.txt': ['116434756.txt'], '116461244.txt': ['116467442.txt'], '116470163.txt': ['116419217.txt'], '116470188.txt': ['116460881.txt'], '116448166.txt': ['116441120.txt'], '116446771.txt': ['116469121.txt', '116468969.txt'], '116443386.txt': ['116471744.txt'], '116463646.txt': ['116458466.txt'], '116465550.txt': ['116452566.txt'], '116443385.txt': ['116464203.txt'], '116447708

In [ ]:
! cat start_data/116446771.txt

        Справа № 279/372/24             Провадження № 1-кс/279/105/24 
                                                                                                                                  
         У Х В А Л А 
                               
19 січня 2024 року 

Слідчий суддя Коростенського міськрайонного суду Житомирської області ОСОБА_1 , з секретарем ОСОБА_2 , розглянувши клопотання слідчого СВ Коростенського РУП ГУНП в Житомирській області  ОСОБА_3  по кримінальному провадженню №12024060490000036 від 16.01.2024 року за ст.185 ч.4 КК України про арешт майна,   
          
В С Т А Н О В И В:

Слідчий Коростенського РУП ГУНП в Житомирській області звернувся з клопотанням про арешт майна, в якому вказав, що 27.12.2023 близько 12 години в магазині «Єва-710» ТОВ «Руш» за адресою: Житомирська область, м. Коростень, вул. Базарна Площа, 1, в період дії воєнного стану, невстановлена особа, таємно викрала гігієничні засоби для догляду за шкірою обличчя та кухонні прилади, на заг

In [15]:
! cat start_data/116469121.txt

Провадження № 1-кс/734/64/24  Справа № 734/195/24




У х в а л а
іменем України

22 січня 2024 року |смт Козелець|

                                                               
       Слідчий суддя Козелецького районного суду Чернігівської області ОСОБА_1 , за участю секретаря судових засідань ОСОБА_2 , розглянувши у судовому засіданні в приміщенні зали суду в смт. Козелець клопотання дізнавача у кримінальному провадженні – т.в.о начальника сектору дізнання ВП №1 ЧРУП ГУНП в Чернігівській області капітана поліції ОСОБА_3 про накладення арешту на майно по кримінальному провадженню №12024275460000003, відомості про яке внесені до Єдиного реєстру досудових розслідувань 01.01.2024 року, за ознаками кримінального правопорушення (проступку), передбаченого ч. 1 ст. 309 КК України, -

В С Т А Н О В И В:

       дізнавач у кримінальному провадженні – т.в.о начальника сектору дізнання ВП №1 ЧРУП ГУНП в Чернігівській області капітан поліції ОСОБА_3 звернулася до суду в рамках кримінального пр